In [66]:
from numba import njit,jit,prange

import numpy as np
#from heisenberg.wavefun import psi


#@jit(nopython=True)
def computetab(tab,conf,L):
    Lambda = L / 4.0
    e0 = Lambda+L* (np.log(2.0) - 0.25)
    diag = 0
    el = 0
    for i in range(L):
        j = np.mod(i+1, L)
        if conf[i]!= conf[j]:
            tab[i] =True
            el = el - 0.5#i segni sono giusti?
            diag = diag - 0.25
        else:
            diag = diag + 0.25
            tab[i] =False # no move is possible
    el+=diag
    bx=Lambda-el
    return bx/e0 , tab, el,diag
#@jit(nopython=True)
def update(conf,tab,iout,el,diag):
    L=conf.shape[0]
    Lambda=L/4.0
    e0 =Lambda+ L* (np.log(2.0) - 0.25)
    #diag=0
    iback=np.mod(iout-1,L)
    if conf[iback]!= conf[iout]:
        tab[iback] =True
        el = el - 1
        diag = diag - 0.5
    else:
        diag = diag + 0.5
        tab[iback] =False # no move is possible
        el+=1
    jout=np.mod(iout+1,L)
    ifor=np.mod(jout+1,L)
    if conf[ifor]!= conf[jout]:
        tab[jout] =True
        el = el - 1
        diag = diag - 0.5
    else:
        diag = diag + 0.5
        tab[jout] =False # no move is possible
        el+=1
    #el+=diag
    #bx=Lambda-el
    return   tab, el,diag

#@njit(parallel=True)
def dmc_heis(L,nw,conf,tab,bx,el,diag,nsteps):
#accumulated weight initialized to one
    wconf = np.ones(nw)
    Lambda = L / 4.0
#parallelized loop on walkers
    for i in range(nsteps):
        for w in range(nw):
            #accumulate the weight
            wconf[w]*=bx[w]
            #random move
            r=np.random.rand()
            gn=np.zeros(L+1)
            gn[0]=Lambda-diag[w]
            for j in range(L):
                if tab[w,j]: gn[j+1]=0.5
            ztry=r*np.sum(gn)
            ntry=gn[0]
            iout=-1
            while(ntry<ztry):
                iout+=1
                ntry+=gn[iout+1]
                if iout>L:
                    print('molto male')
                    break
            jout=np.mod(iout+1, L)
            #non diag move if iout != -1
            if iout!=-1:
                conf[w,iout]=not(conf[w,iout])
                conf[w,jout]=not(conf[w,jout])
                tab[w], bx[w] , el[w], diag[w] = update(conf[w],tab[w],iout,el[w],diag[w])
    #energy weighted average before branching
    #energy = np.sum(el*wconf)/np.sum(wconf)
    #here we call the branching
    #nrand = np.random.rand()
    #jbra,weight, surv = branching(nw,wconf,nrand)
    #conf, tab, diag, bx, el = reshuff(nw, jbra, conf, tab, diag, bx, el)
    return conf, tab#,  weight , energy , el, diag, bx, surv
    

In [67]:
#example
L=4
tab=np.empty(L,dtype=bool)

conf=[np.mod(i,2)==0 for i in range(L)]
conf=np.array(conf,dtype=bool)
print("stato=",conf)
bx,tab,el,diag=computetab(tab,conf,L)
print("diag",diag)
print("tab iniziale=",tab)
print("energia locale= ",el)
iout=0
conf[0]=np.invert(conf[0])
conf[1]=np.invert(conf[1])

print("configurazione post",conf)
print("tab, el e diag= ",update(conf,tab,iout,el,diag))

nw=1
nsteps=10
print(dmc_heis(L,nw,conf,tab,bx,el,diag,nsteps))

stato= [ True False  True False]
diag -1.0
tab iniziale= [ True  True  True  True]
energia locale=  -3.0
configurazione post [False  True  True False]
tab, el e diag=  (array([ True, False,  True, False]), -1.0, 0.0)


IndexError: invalid index to scalar variable.